In [13]:
import random
import shutil
import os
import numpy as np
import cv2 #Currently unused
from skimage.feature import hog #Need to install scikit-image for this -- ended up using HOG from here
from sklearn.preprocessing import MinMaxScaler
from sklearn import svm


test_apple_dir = "model_data/Test/Apples"
test_orange_dir = "model_data/Test/Oranges"

train_apple_dir = "model_data/Train/Apples"
train_orange_dir = "model_data/Train/Oranges"

RAW_DATASET_PATH = "raw_dataset"
RAW_TRAIN_DATASET_PATH = os.path.join(RAW_DATASET_PATH, "Training")
RAW_TEST_DATASET_PATH = os.path.join(RAW_DATASET_PATH, "Test")
DATASET_PATH = "model_data"
TRAIN_DATASET_PATH = os.path.join(DATASET_PATH, "Train")
TEST_DATASET_PATH = os.path.join(DATASET_PATH, "Test")


In [14]:
# ONLY RUN CELL IF NEEDED TO CLEAR ALL THE MODEL_DATA
print(os.getcwd())

if os.path.exists(test_apple_dir):
    shutil.rmtree(test_apple_dir)

if os.path.exists(test_orange_dir):
    shutil.rmtree(test_orange_dir)

if os.path.exists(train_apple_dir):
    shutil.rmtree(train_apple_dir)

if os.path.exists(train_orange_dir):
    shutil.rmtree(train_orange_dir)

if os.path.exists("model_data/TestBinaries/Apples"):
    shutil.rmtree("model_data/TestBinaries/Apples")
if os.path.exists("model_data/TestBinaries/Oranges"):
    shutil.rmtree("model_data/TestBinaries/Oranges")

os.mkdir(test_apple_dir)
os.mkdir(test_orange_dir)
os.mkdir(train_apple_dir)
os.mkdir(train_orange_dir)
os.mkdir("model_data/TestBinaries/Oranges")
os.mkdir("model_data/TestBinaries/Apples")

if (os.path.isfile(os.path.join(TRAIN_DATASET_PATH, "apple_out.csv"))):
    os.remove(os.path.join(TRAIN_DATASET_PATH, "apple_out.csv"))

if (os.path.isfile(os.path.join(TRAIN_DATASET_PATH, "orange_out.csv"))):
    os.remove(os.path.join(TRAIN_DATASET_PATH, "orange_out.csv"))

if os.path.isfile("model.h"):
    os.remove("model.h")




/home/pbhagava/Desktop/EdgeTestbed/edge_testbed/V1/src/sw/test_images


In [15]:
def copyImages(directory, destination, num_files=-1):
    files = os.listdir(directory)
    ret = 0
    if num_files == -1:
        ret = files
    else:
        ret = random.sample(files, num_files)
    for fl in ret:
        path = os.path.join(directory, fl)
        shutil.copy(path, destination)
    return ret

apple_names = ["Apple Red 1"]

#Generate Train Image Set
orange_files = copyImages(os.path.join(RAW_TRAIN_DATASET_PATH, "Orange"), os.path.join(TRAIN_DATASET_PATH, "Oranges"))
NUM_ORANGE_IMAGES = len(orange_files)
NUM_APPLE_IMAGES_PER_APPLE = NUM_ORANGE_IMAGES // len(apple_names)
for apple_type in apple_names:
    copyImages(os.path.join(RAW_TRAIN_DATASET_PATH, apple_type), os.path.join(TRAIN_DATASET_PATH, "Apples"), NUM_APPLE_IMAGES_PER_APPLE)
print("Generated training set!")
print(f"Number of Orange images: {NUM_ORANGE_IMAGES}")
print(f"Number of Apple images: {NUM_APPLE_IMAGES_PER_APPLE * len(apple_names)} | {NUM_APPLE_IMAGES_PER_APPLE} images per apple type")

#Generate Test Image Set
orange_test_files = copyImages(os.path.join(RAW_TEST_DATASET_PATH, "Orange"), os.path.join(TEST_DATASET_PATH, "Oranges"))
NUM_TEST_ORANGE_IMAGES = len(orange_test_files)
NUM_APPLE_TEST_IMAGES_PER_APPLE = NUM_TEST_ORANGE_IMAGES // len(apple_names)
for apple_type in apple_names:
    copyImages(os.path.join(RAW_TEST_DATASET_PATH, apple_type), os.path.join(TEST_DATASET_PATH, "Apples"), NUM_APPLE_TEST_IMAGES_PER_APPLE)
print("\nGenerated testing set!")
print(f"Number of Orange images: {NUM_TEST_ORANGE_IMAGES}")
print(f"Number of Apple images: {NUM_APPLE_TEST_IMAGES_PER_APPLE * len(apple_names)} | {NUM_APPLE_TEST_IMAGES_PER_APPLE} images per apple type")
print(f"Total number of test images: {NUM_APPLE_TEST_IMAGES_PER_APPLE * len(apple_names) + NUM_TEST_ORANGE_IMAGES}")

#Generate Test Binaries Set 
def generate_binaries(dir, fruit, dest):
    files = os.listdir(dir)
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        dst = os.path.join(dest, f"{fruit}-{n}.txt")
        os.system(f'ffmpeg -i {pth} -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e \'1/2 "%u " "\\n"\' -v > {dst}')
    return

generate_binaries(test_orange_dir, "orange", "model_data/TestBinaries/Oranges")
generate_binaries(test_apple_dir, "apple", "model_data/TestBinaries/Apples")
print("\nGenerated Test Binaries!")


Generated training set!
Number of Orange images: 479
Number of Apple images: 479 | 479 images per apple type

Generated testing set!
Number of Orange images: 160
Number of Apple images: 160 | 160 images per apple type
Total number of test images: 320

Generated Test Binaries!


In [327]:
# Upscale/Rescale Images to 320x240
def convertTo320x240(dir, verbose=False):
    files = os.listdir(dir)
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        dst = os.path.join(dir, f"resized-{fl}")
        os.system(f"convert {pth} -resize 320x240 -background white -gravity center -extent 320x240 {dst}")
        if verbose:
            print(f"ran convert {pth} -resize 320x240 -background white -gravity center -extent 320x240 {dst}")
    return

convertTo320x240(test_apple_dir)
convertTo320x240(test_orange_dir)
convertTo320x240(train_apple_dir)
convertTo320x240(train_orange_dir)



In [328]:
# str_ = 'ffmpeg -i converted-230_100.jpg -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e \'1/2 "%u " "\\n"\' -v > test.txt'
# print(str_)
# 
# ffmpeg -i converted-230_100.jpg -vcodec rawvideo -f rawvideo -pix_fmt rgb565 pipe:1 | hexdump -e '1/2 "%u " "\n"' -v > test.txt

def convertTo565(dir, fruit, to_csv=False, verbose=False):
    files = os.listdir(dir)
    for n, fl in enumerate(files):
        pth = os.path.join(dir, fl)
        dst = os.path.join(dir, f"565-{fruit}-{n}.jpg")
        
        os.system(f'convert {pth} -define bmp:subtype=RGB565 {dst}')
        os.remove(pth)
        
        if verbose:
            print(f"ran 565 conversion on {pth} into {dst}")
    return

convertTo565(test_apple_dir, "apple")
convertTo565(test_orange_dir, "orange")
convertTo565(train_apple_dir, "apple")
convertTo565(train_orange_dir, "orange")

In [ ]:
# Convert images into features np arrays
# Also scale X vector


test_image = os.path.join(train_apple_dir, "565-apple-0.jpg")
im = cv2.imread(test_image)
features = np.mean(np.mean(im, axis=0), axis=0)
features_length = features.shape[0]


def get_features(image_dir, label):
    # image_dir - image directory
    # functions takes in directory of images to generate features using HOG and specified label
    # Outputs features, labels
    X = np.empty(features_length)
    y = int(np.empty(1))
    for fl in os.listdir(image_dir):
        im = cv2.imread(os.path.join(image_dir, fl))
        # features = hog(im, orientations=9, pixels_per_cell=(8,8), cells_per_block=(2,2), multichannel=True)
        # print(im.shape)
        features_1 = np.mean(im, axis=0)
        # print(features_1.shape)
        features = np.mean(features_1, axis=0)
        # print(features.shape)
        X = np.vstack([X, features])
        y = np.hstack([y, label])
    return X, y

trainX, trainY = get_features(train_apple_dir, 0)
_X, _y = get_features(train_orange_dir, 1)
trainX = np.vstack([trainX, _X])
trainY = np.hstack([trainY, _y])

print(f"Dimensions of trainX data: {trainX.shape}")
print(f"Dimensions of trainY data: {trainY.shape}")

print(f"NaN check: {np.any(np.isnan(trainX))} (Should be False)")

print(f"Finite check: {np.all(np.isfinite(trainY))} (Should be True)")

Dimensions of trainX data: (960, 3)
Dimensions of trainY data: (960,)
NaN check: False (Should be False)
Finite check: True (Should be True)


In [ ]:
clf = svm.LinearSVC(C=1.)
clf.fit(trainX, trainY)

/usr/local/lib64/python3.6/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [ ]:
testX, testY = get_features(test_apple_dir, 0)
__X, __y = get_features(test_orange_dir, 1)
testX = np.vstack([testX, __X])
testY = np.hstack([testY, __y])

clf.score(testX, testY)

0.9968944099378882

In [ ]:
from sklearn.svm import SVC
from sklearn_porter import Porter
import sys

porter = Porter(clf, language='c')
c_code = porter.export()
original_stdout = sys.stdout
with open('model.h', 'w') as f:
    sys.stdout = f # Change the standard output to the file we created.
    print(c_code)
    sys.stdout = original_stdout # Reset the standard output to its original value

**Ignore further cells**

In [ ]:
# import cv2
# import numpy as np
# # def get_SURF_features(dir):
# #     files = os.listdir(dir)
# #     for n, fl in enumerate(files):
# #         pth = os.path.join(dir, fl)
# #         dst = os.path.join(dir, f"SURF_{fl}")

# #         im = cv2.imread(pth)
# #         # im_2 = cv2.cvtColor(im, cv2.COLOR_BGR5652RGB)
        
# #         if im is None:
# #             os.error("We have a problem, imread isn't working")

# #         surf = cv2.xfeatures2d.SURF_create(400)
# #         keypoints, descriptors = surf.detectAndCompute(im, None)

# #         print(len(keypoints))
# #         print(descriptors.shape)
        
# #     return

# # get_SURF_features(test_apple_dir)
# # get_SURF_features(test_orange_dir)
# # get_SURF_features(train_apple_dir)
# # get_SURF_features(train_orange_dir)

# def get_ORB_features(dir):
#     files = os.listdir(dir)
#     # combined_data = np.array()

#     for n, fl in enumerate(files):
#         pth = os.path.join(dir, fl)
#         dst = os.path.join(dir, f"SURF_{fl}")

#         im = cv2.imread(pth)
#         # im_2 = cv2.cvtColor(im, cv2.COLOR_BGR5652RGB)
        
#         if im is None:
#             os.error("We have a problem, imread isn't working")

#         orb = cv2.ORB_create()
#         keypoints, descriptors = orb.detectAndCompute(im, None)

#         features = descriptors[:20, :]
#         print(features.shape)

#         # print(fl)
#         # print(descriptors.shape)
#         # print(type(descriptors))
        
#     return

# get_ORB_features(test_apple_dir)
# get_ORB_features(test_orange_dir)
# get_ORB_features(train_apple_dir)
# get_ORB_features(train_orange_dir)

In [ ]:
# import pandas as pd
# import csv


# # def convertToCSV(dir):
# #     files = os.listdir(dir)
# #     for n, fl in enumerate(files):
# #         pth = os.path.join(dir, fl)
# #         print("Reading ", pth)
# #         dst = os.path.join(dir, f"CSV_{fl}")
# #         p_csv = pd.read_csv(pth)
# #         p_csv.to_csv(dst)

# def convertToCSV(dir, fruit):
#     files = os.listdir(dir)
#     lst = []
#     for n, fl in enumerate(files):
#         pth = os.path.join(dir, fl)
#         # print("Reading ", pth)
#         p_file = open(pth)
#         p_lst = []
#         for line in p_file:
#             f_line = line[:-2]
#             f_line = int(f_line)
#             p_lst.append(f_line)
#         p_file.close()
#         # print(p_lst[:50])
#         lst.append(p_lst)
#         # print(len(p_lst))
#     # print(len(lst))
#     dst = os.path.join(TRAIN_DATASET_PATH, f"{fruit}_out.csv")
#     with open(dst, "w", newline="") as f:
#         writer = csv.writer(f)
#         writer.writerows(lst)

# convertToCSV(train_apple_dir, "apple")
# convertToCSV(train_orange_dir, "orange")

        


In [ ]:
# import pandas as pd

# df_apple = pd.read_csv(os.path.join(TRAIN_DATASET_PATH, "apple_out.csv"), header=None)
# df_orange = pd.read_csv(os.path.join(TRAIN_DATASET_PATH, "orange_out.csv"), header=None)
# print(df_apple.shape)
# print(df_orange.shape)

In [ ]:
# from sklearn.svm import SVC
# from micromlgen import port
# from sklearn_porter import Porter
# import sys

# def trainClassifier(X, y):
#     clf = SVC(kernel='linear',gamma=0.001).fit(X, y)
#     return clf

# clf = trainClassifier(df_combined.iloc[:, 1:], df_combined.iloc[:, 0])
# # c_code = port(clf, classmap={
# #         0: 'apple',
# #         1: 'orange'
# #     })

# porter = Porter(clf, language='c')


# c_code = porter.export()
# # print(c_code)

In [ ]:
# porter = Porter(clf, language='c')


# c_code = porter.export()
# # print(c_code)
# original_stdout = sys.stdout
# with open('model.h', 'w') as f:
#     sys.stdout = f # Change the standard output to the file we created.
#     print(c_code)
#     sys.stdout = original_stdout # Reset the standard output to its original value


In [ ]:
# from PIL import Image
# from scipy import misc
# import imageio

# def convertBMPToBinaryListHelper(img):
#     im = Image.open(img, mode='r')
#     pixels = list(im.getdata())
#     lst = []
#     for pixel in pixels:
#         r = pixel[0] & 0b11111111
#         g = pixel[1] & 0b11111111
#         b = pixel[2] & 0b11111111
#         rgb = ((r & 0b11111000) << 8) | ((g & 0b11111100) << 3) | ((b & 0b11100000) >> 3)
#         bin_data = bin(rgb)[2:]
#         if (len(bin_data) != 16):
#             print("Error! not 16 bits of data here!", bin_data)
#             print(r, g, b)
#             bin_data = bin_data.zfill(16)
#             print(f"Converted to: {bin_data}")
#         # print(bin_data)
#         lst.append(bin_data)
#     return lst

# def convertBMPToBinaryList(dir):
#     files = os.listdir(dir)
#     for fl in files:
#         pth = os.path.join(dir, fl)
#         dst = os.path.join(dir, f"binary-{fl[:-4]}.txt")
#         bin_list = convertBMPToBinaryListHelper(pth)
#         dest_file = open(dst, "w")
#         dest_file.write(str(bin_list))
#         print(f"Converted {pth} to {dst}")
#         os.remove(pth)
    

# flattened_image = convertBMPToBinaryList("model_data/Test/Oranges")
# # test_lst = convertBMPToBinaryListHelper("model_data/Test/Oranges/orange8.bmp")